# 필요한 모듈 설치 및 임포트

In [6]:
%pip install selenium==4.8.2 beautifulsoup4 pandas numpy webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [7]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


# Selenium을 이용하여 브라우저 열기, 클릭, 페이지 이동, 닫기

In [8]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)

# 무신사 코디맵 크롤링

In [32]:
driver = webdriver.Chrome(options=chrome_options)

codimap_list = []
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

# 페이지를 가장 아래까지 스크롤하는 방법

interval = 1 # 1초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")

# 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height

# 해당 Xpath 정보를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
# elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl']         # 카테고리, 이름, 날짜, 조회수, 댓글수
values = []                                             # 정보를 담을 빈 리스트 생성
soup = BeautifulSoup(driver.page_source, 'html.parser')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) # 해당 조건에 맞는 정보들을 list로 반환
# 각 <li> 태그를 순회하면서 데이터 추출
for i, row in enumerate(data_rows):
    print('{0} {1}번째 코디맵정보 크롤링'.format(0, i+1))
    blank = []

    # 코디맵 카테고리 추출
    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})
    if codimap_category:
        codimap_category = codimap_category.get_text().strip()
        blank.append(codimap_category)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 이름 추출
    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})
    if codimap_title:
        codimap_title = codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 날짜 추출
    codimap_date = row.find('span', attrs={'class':'post-information__text'})
    if codimap_date:
        codimap_date = codimap_date.get_text().strip()
        blank.append(codimap_date)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 조회수 추출
    views = row.find_all('span', attrs={'class':'post-information__text'})[1]
    if views:
        views = views.get_text().strip().split()[1]
        blank.append(views)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 댓글수 추출
    comment_numbers = row.find_all('span', attrs={'class':'post-information__text'})[2]
    if comment_numbers:
        comment_numbers = comment_numbers.get_text().strip().split()[1]
        blank.append(comment_numbers)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue
    # list의 i번째에 해당하는 코디맵 이미지의 xpath를 가져옴
    xpath = '/html/body/div[3]/div[2]/form/div[4]/div/ul/li[' + str(i+1) + ']/div[1]/a/div/img'
    # 해당 xpath를 가진 이미지를 클릭함
    codi_button = driver.find_element(By.XPATH, xpath)
    codi_button.click()
    # 로딩 대기 0.1초(느리면 줄여도 됨) (중간에 몇개만 추출 실패하면 이거 늘려야 함)
    time.sleep(0.1)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')# 가끔 soup이 이상해져서 추가함
    # 코디맵 설명과 해시태그 추출
    codimap_info = soup.find('div', attrs={'class':'styling_content codimap-contents'})  
    codimap_explain = codimap_info.find('p', attrs={'class':'styling_txt'})
    if codimap_explain:
        codimap_explain = codimap_explain.get_text().strip()
        blank.append(codimap_explain)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생1'.format(i+1))
        continue
    
    codimap_hashtag = codimap_info.find_all('a', attrs={'class':'ui-tag-list__item'})
    for j in range(len(codimap_hashtag)):
        if codimap_hashtag[j]:
            codimap_hashtag[j] = codimap_hashtag[j].get_text().strip()
        else:
            codimap_hashtag[j] = 'Something is wrong'
            print('{}번째 코디맵 해시태그 가져올때 문제발생'.format(i+1))
            continue    
    if codimap_hashtag:
        blank.append(codimap_hashtag)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생2'.format(i+1))
        continue
    # 코디맵 이미지 URL 추출
    codimap_imgurl = soup.find('img', attrs={'class':'photo'})
    if codimap_imgurl:
        codimap_imgurl = codimap_imgurl.get('src').strip() # 이미지 url은 text가 아니라 get('src')로 추출
        blank.append(codimap_imgurl)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생3'.format(i+1))
        continue
    driver.back() # 뒤로가기 버튼 클릭하여 원래 페이지로 돌아가기

    values.append(blank)
    print('---------------------------------------------------')
time.sleep(1)

df = pd.DataFrame(values, columns = columns)
codimap_list.append(df)


0 1번째 코디맵정보 크롤링
---------------------------------------------------
0 2번째 코디맵정보 크롤링
---------------------------------------------------
0 3번째 코디맵정보 크롤링
---------------------------------------------------
0 4번째 코디맵정보 크롤링
---------------------------------------------------
0 5번째 코디맵정보 크롤링
---------------------------------------------------
0 6번째 코디맵정보 크롤링
---------------------------------------------------
0 7번째 코디맵정보 크롤링
---------------------------------------------------
0 8번째 코디맵정보 크롤링
---------------------------------------------------
0 9번째 코디맵정보 크롤링
---------------------------------------------------
0 10번째 코디맵정보 크롤링
---------------------------------------------------
0 11번째 코디맵정보 크롤링
---------------------------------------------------
0 12번째 코디맵정보 크롤링
---------------------------------------------------
0 13번째 코디맵정보 크롤링
---------------------------------------------------
0 14번째 코디맵정보 크롤링
---------------------------------------------------
0 15번째 코디맵정보 크롤링
--------------------------

In [33]:
df_ = pd.concat(codimap_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,21.07.02,"17,344",50,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,"[#톤온톤, #뉴트럴톤, #캠퍼스, #피크닉, #와이드핏, #오버핏, #꾸안꾸, #...",//image.msscdn.net/images/codimap/detail/4035/...
1,캐주얼,블랙 매니아,21.02.23,"36,378",47,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,"[#봄, #캐주얼, #라퍼지스토어, #다이아몬드 레이라, #나이키, #엠프렌즈, #...",//image.msscdn.net/images/codimap/detail/1969/...
2,캐주얼,오늘의 코디,22.03.22,"13,444",44,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,"[#로고플레이, #오버핏, #스티치, #캠퍼스, #봄, #캐주얼, #유니폼브릿지, ...",//image.msscdn.net/images/codimap/detail/13814...
3,댄디,추천 댄디 룩,21.09.15,"21,639",32,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,"[#데이트, #그래픽, #레이어드룩, #가을, #댄디, #엘무드, #뮤지엄바이비컨,...",//image.msscdn.net/images/codimap/detail/6746/...
4,캐주얼,릴랙스 룩,21.02.23,"24,068",25,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,"[#봄, #캐주얼, #카시오, #코닥, #제로, #와릿이즌, #뉴발란스, #커버낫,...",//image.msscdn.net/images/codimap/detail/1974/...
5,스트릿,힙하다 힙해,21.02.03,"36,121",24,레더 블루종 점퍼에 오버사이즈 니트와 조거 팬츠를 매치하고 블루 컬러 볼캡으로 멋을...,"[#봄, #스트릿, #제로, #월간, #라퍼지스토어, #키치오브제, #블론드나인, ...",//image.msscdn.net/images/codimap/detail/1701/...
6,댄디,데이트 코디,21.12.24,"11,010",19,깔끔한 무드의 니트에 데님 팬츠를 스타일링하고 스니커즈로 마무리한 댄디 룩,"[#남친룩, #레더, #데이트, #테이퍼드핏, #오버핏, #소개팅, #스티치, #겨...",//image.msscdn.net/images/codimap/detail/10584...
7,캐주얼,심플한 코디,21.07.30,"41,388",18,로고 프린팅이 돋보이는 반팔 티셔츠와 스웨트 팬츠를 매치하고 스니커즈로 완성한 이지...,"[#로고플레이, #편안한, #원마일웨어, #와이드핏, #여름, #캐주얼, #리, #...",//image.msscdn.net/images/codimap/detail/5053/...
8,댄디,훈훈 남친 룩,21.03.24,"20,037",17,체크 패턴이 돋보이는 블레이저에 셔츠와 데님 팬츠를 더하고 담백한 구두로 마무리한 ...,"[#봄, #댄디, #모디파이드, #반, #락포트, #수아레, #슈트/블레이저 재킷,...",//image.msscdn.net/images/codimap/detail/2392/...
9,댄디,서머 데이트 룩,21.07.26,"8,386",17,깔끔한 셔츠와 니트 베스트를 레이어드하고 클러치 백으로 완성한 댄디 룩,"[#미니멀, #데이트, #오버핏, #로고플레이, #하객, #화사한, #여름, #댄디...",//image.msscdn.net/images/codimap/detail/4851/...


In [34]:
df_.shape

(60, 8)

# Dataframe을 csv 파일로 저장

In [35]:
df_.to_csv('무신사_코디맵_크롤링_도전1.csv', encoding = 'utf-8-sig') # 한글이 들어간 정보는 글자가 깨질 수 있어 encoding을 다음과 같이 지정하여 저장